# Tutorial

## 1. Database Design and Setup
First, we design our database.

We need two tables: passengers and flights.

A passenger is booked on a flight, so we will link them using the flight_id.

- flights: flight_id will be the PRIMARY KEY.
- passengers: passenger_id is the PRIMARY KEY, and flight_id is a FOREIGN KEY that points to the flights table.

In [1]:
import sqlite3

# Connect to a database (or create one if it doesn't exist)
conn = sqlite3.connect('airport.db')
cursor = conn.cursor() # is like a controller or handler for talking to the database / cursor() = talking to the database

In [2]:
# Create the updated tables / # leave code lines DROP TABLE... out will create an ERROR since tables already exist in the database, it will raise an error:
cursor.executescript("""
    DROP TABLE IF EXISTS passengers;    --- only for learning 
    DROP TABLE IF EXISTS flights;       --- only for learning

    CREATE TABLE flights (
        flight_id INTEGER PRIMARY KEY,
        flight_number TEXT NOT NULL,
        destination TEXT NOT NULL,
        departure_time TEXT, -- TEXT for ISO formatted datetime (YYYY-MM-DD HH:MM:SS)
        status TEXT
    );

    CREATE TABLE passengers (
        passenger_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        assigned_seat TEXT,
        flight_id INTEGER,
        FOREIGN KEY (flight_id) REFERENCES flights (flight_id)
    );
""")

print("Updated database and tables created successfully.")
conn.commit() # saves all changes i made to the database during the current connection

Updated database and tables created successfully.


## 2. Inserting Data

Adding some flights and passengers.

In [3]:
# Insert data into the flights table
flights_data = [
    (1, 'BA2490', 'London', '2025-05-26 14:00:00', 'On Time'),
    (2, 'LH1020', 'Frankfurt', '2025-05-26 15:30:00', 'Delayed'),
    (3, 'AF1234', 'Paris', '2025-05-26 12:15:00', 'On Time'),
    (4, 'IB3160', 'Madrid', '2025-05-27 09:00:00', 'Cancelled'),
    (5, 'TP1111', 'Lisbon', '2025-05-27 11:00:00', 'On Time') # A flight with no passengers yet
]
cursor.executemany("INSERT INTO flights VALUES (?, ?, ?, ?, ?)", flights_data)

# Insert data into the passengers table
passengers_data = [
    (101, 'Alice Johnson', '14A', 1),
    (102, 'Bob Williams', '21C', 2),
    (103, 'Charlie Brown', None, 1),
    (104, 'Diana Miller', '18B', 3),
    (105, 'Eve Davis', '22F', 2),
    (106, 'Frank White', '14B', 1)
]
cursor.executemany("INSERT INTO passengers VALUES (?, ?, ?, ?)", passengers_data)

print("New data inserted successfully.")
conn.commit()

New data inserted successfully.


## 3. Filtering with WHERE and IN

The WHERE clause is used to filter records based on specific conditions.

In [ ]:
# WHERE with a number: Find flights with a specific ID
cursor.execute("SELECT * FROM flights WHERE flight_id = 2;") #cursor.execute() is sending a SQL query to the SQL database
print("\nFlight with ID 2:", cursor.fetchone()) # fetchone() returns the first row of the result set


Flight with ID 2: (2, 'LH1020', 'Frankfurt', '2025-05-26 15:30:00', 'Delayed')


In [ ]:
# WHERE with text: Find all flights going to London
cursor.execute("SELECT * FROM flights WHERE destination = 'London';")
print("\nFlights to London:", cursor.fetchall()) # fetchall() returns all rows of the result set


Flights to London: [(1, 'BA2490', 'London', '2025-05-26 14:00:00', 'On Time')]


In [6]:
# WHERE with datetime: Find flights departing after a certain time
cursor.execute("SELECT * FROM flights WHERE departure_time > '2025-05-26 15:00:00';")
print("\nFlights after 3 PM on May 26th:", cursor.fetchall())


Flights after 3 PM on May 26th: [(2, 'LH1020', 'Frankfurt', '2025-05-26 15:30:00', 'Delayed'), (4, 'IB3160', 'Madrid', '2025-05-27 09:00:00', 'Cancelled'), (5, 'TP1111', 'Lisbon', '2025-05-27 11:00:00', 'On Time')]


In [7]:
# WHERE with NULL: Find passengers without an assigned seat
cursor.execute("SELECT name FROM passengers WHERE assigned_seat IS NULL;")
print("\nPassengers with no seat:", cursor.fetchall())


Passengers with no seat: [('Charlie Brown',)]


In [ ]:
# IN clause: Find flights to either Paris or Madrid
cursor.execute("SELECT * FROM flights WHERE destination IN ('Paris', 'Madrid');")
print("\nFlights to Paris or Madrid:", cursor.fetchall()) # fetchall() returns all rows of the result set


Flights to Paris or Madrid: [(3, 'AF1234', 'Paris', '2025-05-26 12:15:00', 'On Time'), (4, 'IB3160', 'Madrid', '2025-05-27 09:00:00', 'Cancelled')]


## 4. Understanding Join Types

Joins are used to combine rows from two or more tables based on a related column.

INNER JOIN: Selects records that have matching values in both tables. This is the most common join.

LEFT JOIN: Selects all records from the left table (flights), and the matched records from the right table (passengers).

If there is no match, the result is NULL on the right side. This is useful for finding things like flights with no passengers.

In [ ]:
# INNER JOIN: Show passengers and their flight information / usually ids from both tables are used to join
inner_join_query = """
    SELECT p.name, f.flight_number, f.destination
    FROM passengers p
    INNER JOIN flights f ON p.flight_id = f.flight_id;
"""
cursor.execute(inner_join_query) #cursor.execute() is sending a SQL query to the SQL database
print("\nPassengers with their flights (INNER JOIN):", cursor.fetchall()) # fetchall() returns all rows of the result set


Passengers with their flights (INNER JOIN): [('Alice Johnson', 'BA2490', 'London'), ('Bob Williams', 'LH1020', 'Frankfurt'), ('Charlie Brown', 'BA2490', 'London'), ('Diana Miller', 'AF1234', 'Paris'), ('Eve Davis', 'LH1020', 'Frankfurt'), ('Frank White', 'BA2490', 'London')]


In [ ]:
# LEFT JOIN: Show ALL flights and the passengers on them, if any
left_join_query = """
    SELECT f.flight_number, f.destination, p.name
    FROM flights f
    LEFT JOIN passengers p ON f.flight_id = p.flight_id;
"""
cursor.execute(left_join_query)
print("\nAll flights and their passengers (LEFT JOIN):", cursor.fetchall())


All flights and their passengers (LEFT JOIN): [('BA2490', 'London', 'Alice Johnson'), ('BA2490', 'London', 'Charlie Brown'), ('BA2490', 'London', 'Frank White'), ('LH1020', 'Frankfurt', 'Bob Williams'), ('LH1020', 'Frankfurt', 'Eve Davis'), ('AF1234', 'Paris', 'Diana Miller'), ('IB3160', 'Madrid', None), ('TP1111', 'Lisbon', None)]


## 5. Filtering Groups with HAVING

We've used GROUP BY to group rows.

But what if you want to filter these groups? You can't use WHERE because WHERE filters rows before they are grouped.

Instead, you use the HAVING clause. HAVING filters groups after the aggregation function has been applied.

Rule: WHERE filters rows, HAVING filters groups

In [11]:
# Find flights that have MORE THAN 1 passenger.
# 1. JOIN the tables
# 2. GROUP the results by flight
# 3. COUNT the passengers in each group
# 4. HAVING filters these groups to keep only those with a count > 1

query = """
    SELECT
        f.flight_number,
        COUNT(p.passenger_id) AS passenger_count
    FROM
        flights f
    JOIN
        passengers p ON f.flight_id = p.flight_id
    GROUP BY
        f.flight_id
    HAVING
        passenger_count > 1;
"""

cursor.execute(query)
print("Flights with more than 1 passenger:", cursor.fetchall())

Flights with more than 1 passenger: [('BA2490', 3), ('LH1020', 2)]


## 6. A Critical Warning: SQL Injection ⚠️

SQL Injection is a major security vulnerability.

It happens when an attacker can manipulate your SQL queries by inserting malicious code into the data your program uses.

NEVER build queries using string formatting like f-strings or +.

TO READ: [SQL Injection in python](https://realpython.com/prevent-python-sql-injection/)

In [27]:
# Very Important: Close the connection
conn.close()

## 7. SQL Alchemy

In [38]:
 !pip install sqlalchemy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\namro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [39]:
from sqlalchemy import create_engine # create_engine is a function that lets you set up a connection to a database (like SQLite, MySQL, PostgreSQL, etc.) using SQLAlchemy

engine = create_engine("sqlite:///airport.db")

In [40]:
from sqlalchemy.ext.declarative import declarative_base # declarative_base is a function that defines database tables as python classes

# Base class for declarative models
Base = declarative_base()

C:\Users\namro\AppData\Local\Temp\ipykernel_13512\3627678555.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [41]:
# This code sets up the database schema in Python. It tells Python and SQLAlchemy how the database tables should look and how they relate to each other.

from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship, sessionmaker


# Define the Flights table
class Flight(Base):
    __tablename__ = 'flights'

    flight_id = Column(Integer, primary_key=True)
    flight_number = Column(String, nullable=False)
    destination = Column(String, nullable=False)
    departure_time = Column(String)  # Stored as ISO formatted text
    status = Column(String)

    # One-to-many relationship to passengers
    passengers = relationship("Passenger", back_populates="flight")

# Define the Passengers table
class Passenger(Base):
    __tablename__ = 'passengers'

    passenger_id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    assigned_seat = Column(String)
    flight_id = Column(Integer, ForeignKey('flights.flight_id'))

    # Relationship to flight
    flight = relationship("Flight", back_populates="passengers")

In [42]:
# Create the tables in the database
Base.metadata.drop_all(engine)  # Only for learning/demo purposes
Base.metadata.create_all(engine)

print("Updated database and tables created successfully.")

Updated database and tables created successfully.


In [43]:
from sqlalchemy.orm import sessionmaker

# Set up the session
Session = sessionmaker(bind=engine)
session = Session()

# Insert flights
flights_data = [
    Flight(flight_id=1, flight_number='BA2490', destination='London', departure_time='2025-05-26 14:00:00', status='On Time'),
    Flight(flight_id=2, flight_number='LH1020', destination='Frankfurt', departure_time='2025-05-26 15:30:00', status='Delayed'),
    Flight(flight_id=3, flight_number='AF1234', destination='Paris', departure_time='2025-05-26 12:15:00', status='On Time'),
    Flight(flight_id=4, flight_number='IB3160', destination='Madrid', departure_time='2025-05-27 09:00:00', status='Cancelled'),
    Flight(flight_id=5, flight_number='TP1111', destination='Lisbon', departure_time='2025-05-27 11:00:00', status='On Time')
]

session.add_all(flights_data)

# Insert passengers
passengers_data = [
    Passenger(passenger_id=101, name='Alice Johnson', assigned_seat='14A', flight_id=1),
    Passenger(passenger_id=102, name='Bob Williams', assigned_seat='21C', flight_id=2),
    Passenger(passenger_id=103, name='Charlie Brown', assigned_seat=None, flight_id=1),
    Passenger(passenger_id=104, name='Diana Miller', assigned_seat='18B', flight_id=3),
    Passenger(passenger_id=105, name='Eve Davis', assigned_seat='22F', flight_id=2),
    Passenger(passenger_id=106, name='Frank White', assigned_seat='14B', flight_id=1)
]

session.add_all(passengers_data)
session.commit()

print("New data inserted successfully.")


New data inserted successfully.


In [ ]:
# this code is querying the database to find flights that have more than one passenger.

from sqlalchemy import func

results = (
    session.query( #starts a query to the database
        Flight.flight_number, # selects the flight number to display
        func.count(Passenger.passenger_id).label("passenger_count") # counts the number of passengers (via Passenger.passenger_id) and labels it as "passenger_count"
    ).join(Passenger) # joins the Flight and Passenger tables
    .group_by(Flight.flight_id) # groups the results by flight ID
    .having(func.count(Passenger.passenger_id) > 1) # filters the groups to keep only those with more than 1 passenger
    .all() # executes the query and retrieves all results
)

In [46]:
print("\nFlights with more than one passenger:")
for flight_number, count in results:
    print(f"Flight {flight_number} has {count} passengers.")


Flights with more than one passenger:
Flight BA2490 has 3 passengers.
Flight LH1020 has 2 passengers.


In [47]:
engine.dispose()  # Close the connection to the database

## 8. Read data with Pandas

In [1]:
 !pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\namro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# this code reads data from your SQLite database into Pandas DataFrames using SQLAlchemy.

import pandas as pd
from sqlalchemy import create_engine # from SQLAlchemy to connect to the database

# Connects to airport.db SQLite database using SQLAlchemy
engine = create_engine('sqlite:///airport.db')

# Loads the entire flights table into a a Pandas Dataframe called flights_df
flights_df = pd.read_sql_table('flights', con=engine)

# Read the passengers table into a DataFrame
passengers_df = pd.read_sql_table('passengers', con=engine)

engine.dispose()  # Close the connection to the database

# You can now use flights_df and passengers_df in Python for data analysis, flitering, or visualization.

# ORM stands for Object-Relational Mapping. It lets you use Python classes and objects to interact with a database — instead of writing raw SQL

In [3]:
flights_df

,flight_id,flight_number,destination,departure_time,status
0,1,BA2490,London,2025-05-26 14:00:00,On Time
1,2,LH1020,Frankfurt,2025-05-26 15:30:00,Delayed
2,3,AF1234,Paris,2025-05-26 12:15:00,On Time
3,4,IB3160,Madrid,2025-05-27 09:00:00,Cancelled
4,5,TP1111,Lisbon,2025-05-27 11:00:00,On Time


In [4]:
passengers_df

,passenger_id,name,assigned_seat,flight_id
0,101,Alice Johnson,14A,1
1,102,Bob Williams,21C,2
2,103,Charlie Brown,None,1
3,104,Diana Miller,18B,3
4,105,Eve Davis,22F,2
5,106,Frank White,14B,1


In [ ]:
# this code reads data from your SQLite database into Pandas DataFrames using SQLAlchemy.

# USING sqlite3

import pandas as pd
import sqlite3

# 
conn = sqlite3.connect('airport.db')  # Adjust path if needed

# Read the flights table
flights_df = pd.read_sql_query("SELECT * FROM flights", conn)

# Read the passengers table
passengers_df = pd.read_sql_query("SELECT * FROM passengers", conn)

# Close the connection (optional but good practice)
conn.close()

In [53]:
flights_df

,flight_id,flight_number,destination,departure_time,status
0,1,BA2490,London,2025-05-26 14:00:00,On Time
1,2,LH1020,Frankfurt,2025-05-26 15:30:00,Delayed
2,3,AF1234,Paris,2025-05-26 12:15:00,On Time
3,4,IB3160,Madrid,2025-05-27 09:00:00,Cancelled
4,5,TP1111,Lisbon,2025-05-27 11:00:00,On Time


In [54]:
passengers_df

,passenger_id,name,assigned_seat,flight_id
0,101,Alice Johnson,14A,1
1,102,Bob Williams,21C,2
2,103,Charlie Brown,None,1
3,104,Diana Miller,18B,3
4,105,Eve Davis,22F,2
5,106,Frank White,14B,1


## 9. Resources

The functions COUNT(), SUM(), AVG(), MIN(), and MAX() are the most common aggregation functions.

To learn more about them and others, check out these resources:



W3Schools: Provides easy-to-understand examples for SQL aggregate functions.

https://www.google.com/search?q=https://www.w3schools.com/sql/sql_aggr.asp

TutorialsPoint: A quick reference for SQL aggregate functions.

https://www.tutorialspoint.com/sql/sql-aggregate-functions.htm

---

W3Schools: Learn different JOIN properties

https://www.w3schools.com/sql/sql_join.asp

---

===> VERY IMPORTANT:
[File Based vs SQL Lite vs SQL Alchemy](https://realpython.com/python-sqlite-sqlalchemy/)

---

# Home Work

Homework Exercises: Inventory System

Write the Python code to solve the following problems.

Inventory Database Design: Design a database named inventory.db with two tables:

1. products: product_id, product_name, category, and supplier_id.
2. suppliers: supplier_id, supplier_name, country.
Write the Python code using sqlite3 to create these two tables and make sure to choose the data type and assign primary keys.

Data Population & Analysis:
- Create the database using SQL Alchemy and SQL Lite
- Insert at least three suppliers and five products into the tables you created, ensuring each product has a valid supplier_id.
- Write a Python script that uses a JOIN and GROUP BY to find the number of products supplied by each supplier.

Your script should print the supplier_name and their corresponding product count.